In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

# autor: Renato Farruggio
# date: 2020-06-03

# SAT problem
SAT means satisfiability problem: Given a binary formula, is there (at least) a particular truth assignment for each of its variables such that the whole formula equates to TRUE?

# SAT solver
Generally, a SAT solver can solve the SAT problem. This solver is based on quantum mechanics. However, the circuit this program produces offers no speedup to classical computers.  

This solver is only concerned about whether a solution exists or not. It shouldn't be too hard to extend it to show the actual truth assignment of a solution, but that's secondary.


# Valid inputs
## Definition
A valid input string is consists of at least one predicate ('&' or '|') and is built of:
* Variables ("a", "b", "c", ...): Symbols from the alphabet, in lower case.
* Negation sign ("!" or "NOT"): Has to stand before a variable or before an opening bracket "(" or another "!". "!!" gets replaced by "" automatically.
* Predicates:
* - And ("&" or "AND")
* - OR ("|" or "OR")
* Brackets ("(" and ")"): Have to appear pairwise and must contain exactly one predicate. Between the opening bracket and the proposition, aswell as between the proposition and the closing bracket, there must be exactly one variable or another pair of brackets.
* Spaces (" "): Can be put literally anywhere in the string.
* The brackets that contains the whole formula may be abandoned.

## Examples of valid inputs
* "!a&a"
* "xANDy"
* "!(c&!a)"
* "(a AND b)"
* "b  OR NOT b"
* "(bO  Rc)A NDd"
* "!!!( ( (a OR c) & ( (a|b) & !a ) )  AND (!c | b) )"

## Examples of invalid inputs
* ""
* " "
* a && b
* (b ! AND a)
* (a|b|c)
* (é AND b)

In [ ]:
########                MAIN CELL                 ########
# THIS IS THE ONLY CELL A USER SHOULD BE CONCERNED ABOUT #
## Before using, go to "Kernel" -> "Restart & Run All". ##
###      Also, there is a DRAW CELL right below.       ###

# This will make sure that this entire block gets skipped in the setup process.
try:
    initialized
except NameError:
    initialized = False



if initialized:
    # 0. Decide whether to use a simulator or an actual device. 
    #  Simulator is faster, real device is more fun.
    useSimulator = True

    # 1. Define input string
    inString = "!!!( ( (a OR c) & ( (a|b) & !a ) )  AND (!c | b) )"

    # 2. Build circuit
    (circuit, numVars) = buildCircuit(inString)

    if useSimulator:
        # 3. a) Run circuit on simulator
        runCircuitOnSimulator(circuit, numVars)
    else:
        # 3. b) Run circuit on real device. This will run the circuit on the IBMQ_Melbourne.
        runCircuitOnRealDevice(circuit, numVars)

In [ ]:
###    DRAW CELL    ###

#print(inString)
#circuit.draw()

### Sources: 
* https://towardsdatascience.com/solving-satisfiability-problems-with-grovers-algorithm-quantum-computing-c9d6c177b966

* https://cnot.io/quantum_algorithms/grover/using_grovers_algorithm.html

In [ ]:
from sys import exit

## GLOBAL VARIABLES
charAND = '&'
charOR = '|'
charNOT = '!'
fullAlphabet = "abcdefghijklmnopqrstuvwxyz"

In [ ]:
# returns True if inputString is valid. Return False otherwise.
# return: (valid, alphabet, negatedAlphabet, newstr, numZwischenspeicher)
def validateInput(inputstring) -> (bool, str, str, str, int):
    valid = True
    alphabet = ""
    numVariables = 0
    numZwischenspeicher = 0
    newstr = ""
    tempstr = inputstring.replace(" ", "").replace("AND", charAND).replace("OR", charOR).replace("NOT", charNOT).replace("!!", "")
    
    
        
    #print("Raw input:", inputstring)
    
    openBrackets = 0
    andCounter = 0
    orCounter = 0
    negatedAlphabet = ""
    for pos in range(len(tempstr)):
        char = tempstr[pos]
        #print(pos, char, valid)
        #print("newstr is now", newstr)
        
        if char == charOR or char == charAND:
            newstr += char
            if pos-1 < 0 or pos+1 >= len(tempstr):
                exit(char + " can't be at the beginning or end of the formula!")
            prev = tempstr[pos-1]
            next_ = tempstr[pos+1]
            if prev == charOR or prev == charAND or next_ == charOR or next_ == charAND:
                valid = False
                exit("Invalid formula.")
            continue
        
        if char == '(':
            newstr += '('
            openBrackets += 1
            continue
            
        if char == ')':
            newstr += ')'
            openBrackets -= 1
            numZwischenspeicher += 1
            if openBrackets < 0:
                valid = False
                exit("Invalid formula: Trying to close a non opened bracket.")
            continue
        
        if char == '!':
            if tempstr[pos+1] not in fullAlphabet and tempstr[pos+1] != '(':
                valid = False
                exit("Invalid formula: ! is only allowed before a variable")
            else:
                newstr += charNOT
            continue
        
        if char in fullAlphabet:
            if char not in alphabet:
                alphabet += char
                numVariables += 1
            if newstr[-1:] == charNOT:
                if char.upper() not in negatedAlphabet:
                    negatedAlphabet += char.upper()
                newstr = newstr[:-1] + char.upper()
                continue
            newstr += char
            continue
        
        print("Unknown character:", char)
        
    
    if openBrackets != 0:
        valid = False
        exit("Invalid formula: Did you maybe forget to close a bracket?")
    
    numPredicates = newstr.count('&') + newstr.count('|')
    numOpenBrackets = newstr.count('(')
    numCloseBrackets = newstr.count(')')
    if numOpenBrackets != numCloseBrackets:
        exit("Every bracket has to be closed. Although, if this error occurs, there is something wrong in the code.")
    if numPredicates == numOpenBrackets+1:
        newstr = '(' + newstr + ')'
        numZwischenspeicher += 1
    
    #print()
    #print("valid:", valid)
    #print("alphabet:", alphabet)
    #print("negatedAlphabet:", negatedAlphabet)
    #print("newstr:", newstr)
    #print("numZwischenspeicher:", numZwischenspeicher)
    return (valid, alphabet, negatedAlphabet, newstr, numZwischenspeicher)


In [ ]:
def quantumOr(c1, c2, t, circuit) -> None:
    circuit.ccx(c1,c2,t)
    circuit.cx(c1,t)
    circuit.cx(c2,t)
    circuit.barrier()
    pass

def quantumAnd(c1, c2, t, circuit) -> None:
    circuit.ccx(c1,c2,t)
    circuit.barrier()
    pass

In [ ]:
def buildCircuit(inString) -> (QuantumCircuit, int):
    
    print("Initial formula:", inString)
    
    # This can be used to determine, which truth assignment gets accepted and which one does not.
    fullOutputMeasurement = False

    (valid, alphabet, negatedAlphabet, inputstringFormatted, numZwischenspeicher) = validateInput(inString)
    
    numVariables = len(alphabet)
    numNegated = len(negatedAlphabet)
    numQubits = numVariables+numNegated+numZwischenspeicher
    print("Formatted string:", inputstringFormatted)
    #print(alphabet, negatedAlphabet, numZwischenspeicher)


    if valid:
        var = {}
        negVar = {}
        buffer = {}
        # Setup indices arrays
        for i in range(len(alphabet)):
            var[alphabet[i]] = i
        for i in range(len(negatedAlphabet)):
            negVar[negatedAlphabet[i]] = len(alphabet) + i
        for i in range(numZwischenspeicher):
            buffer["Z"+str(i)] = len(alphabet) + len(negatedAlphabet) + i

        # Define Circuit
        qr = QuantumRegister(numQubits)
        if fullOutputMeasurement:
            cr = ClassicalRegister(numVariables+1)
        else:
            cr = ClassicalRegister(1)
        circuit = QuantumCircuit(qr,cr)

        # Initialize initial hadamard gates
        for i in range(numVariables):
            circuit.h(qr[i])

        # Initialize negated qubits
        for i in range(len(negatedAlphabet)):
            controlIndex = alphabet.index(negatedAlphabet[i].lower())
            targetIndex = negVar[negatedAlphabet[i]]
            circuit.x(qr[targetIndex])
            circuit.cx(controlIndex, targetIndex)

        circuit.barrier()

        ## SETUP CIRCUIT ##

        currentIndex = 0
        maxClauses = numQubits
        formula = inputstringFormatted
        for k in range(numZwischenspeicher):
            to = formula.find(")")
            frm = formula[:to].rfind("(")
            negated = formula[frm-1] == charNOT

            # charAND = '&'
            # charOR  = '|'
            # charNOT = '!'

            # Find predicate
            indexOfPredicate = formula[frm+1:to].find(charAND)
            if indexOfPredicate == -1:
                indexOfPredicate = formula[frm+1:to].find(charOR)
            indexOfPredicate += frm+1

            # Find variables
            varA = formula[frm+1:indexOfPredicate]
            varB = formula[indexOfPredicate+1:to]

            # Define indices
            #  indexOfA
            #  indexOfB
            #  indexOfOut
            indices = []
            for variable in [varA, varB]:
                if variable in alphabet:
                    indices.append(var[variable])    
                elif variable in negatedAlphabet:
                    indices.append(negVar[variable])  
                elif variable[-1] == ' ':
                    indices.append(buffer[variable[:-1]]) 
                else:
                    exit("var not found: " + variable)
            [indexOfA, indexOfB] = indices
            indexOfOut = buffer["Z"+str(currentIndex)]

            # Apply gates to the circuit
            if negated:
                circuit.x(indexOfOut)
            if formula[indexOfPredicate] == charAND:
                quantumAnd(indexOfA, indexOfB, indexOfOut, circuit)
            elif formula[indexOfPredicate] == charOR:
                quantumOr(indexOfA, indexOfB, indexOfOut, circuit)

            # Replace string
            if negated:
                formula = formula[:frm-1] + "Z" + str(currentIndex) + " " + formula[to+1:]
            else:
                formula = formula[:frm] + "Z" + str(currentIndex) + " " + formula[to+1:]

            currentIndex += 1

            if "" == formula[:frm]+formula[to+1:]:
                break
                
                
        # Measure
        if fullOutputMeasurement:
            for i in range(numVariables):
                circuit.measure(qr[i],cr[i])
            circuit.measure(qr[-1],cr[-1])
        else:
            circuit.measure(qr[-1],cr)

    else:
        print("INPUT NOT VALID")

    
    return (circuit, numVariables)
    

In [ ]:
def runCircuitOnSimulator(circuit, numVariables):
    # Run Circuit
    backend_sim = Aer.get_backend('statevector_simulator')
    job_sim = execute(circuit, backend_sim)
    statevec = job_sim.result().get_statevector()

    backend = Aer.get_backend('qasm_simulator')
    shots = 2**numVariables
    results = execute(circuit, backend=backend, shots=shots).result()
    answer = results.get_counts()

    expectedProbability = 1/shots
    actualProbability = int(0 if answer.get('1') is None else answer.get('1'))/shots
    #if not fullOutputMeasurement:
    print('Expected probability:', expectedProbability)
    
    print("Actual Probability", actualProbability)
    print()
    if actualProbability >= expectedProbability / 2:
        print("There is most probably a solution.")
    else:
        print("There is most probably NO solution.")

In [ ]:
def runCircuitOnRealDevice(qc: QuantumCircuit, numVars: int):
    try:
        provider
    except NameError:
        provider = IBMQ.load_account()

    device = provider.get_backend('ibmq_16_melbourne')
    from qiskit.tools.monitor import job_monitor
    
    if numVars > 13:
        exit("Can't run a circuit with more than 13 variables on a real device, because I can't run with that many shots (2^numVariables).")
    
    shots = 2**numVars
    max_credits = 10
    
    job = execute(qc, backend=device, shots=shots, max_credits=max_credits)
    job_monitor(job, interval = 1)
    
    answer = job.result().get_counts(qc)

    expectedProbability = 1/shots
    actualProbability = int(0 if answer.get('1') is None else answer.get('1'))/shots
    #if not fullOutputMeasurement:
    print('Expected probability:', expectedProbability)
    
    print("Actual Probability", actualProbability)
    print()
    if actualProbability >= expectedProbability / 2:
        print("There is most probably a solution.")
    else:
        print("There is most probably NO solution.")

In [ ]:
initialized = True